# Question Answering Model
### A typical question-answering model has two inputs: a natural-language question and a text snippet (such as a news article) providing information to be used for answering the question. The model must then produce an answer: in the simplest possible setup, this is a one-word answer obtained via a softmax over some predefined vocabulary

### Solution: We will set up two independent branches, encoding the text input and the question input as representation vectors; then, concatenate these vectors; and finally, add a softmax classifier on top of the concatenated representations.

In [0]:
from keras.models import Model
from keras import layers
from keras import Input

In [0]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

### Setup reference text branch

In [7]:
text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

Instructions for updating:
Colocations handled automatically by placer.


### Setup Question branch

In [0]:
question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)

### Setup the concatenation node where both branches meet

In [0]:
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

### Add Softmax classifier on top

In [0]:
answer = layers.Dense(answer_vocabulary_size, activation='softmax') (concatenated)

### Compile the model

In [12]:
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

### Training: There are two possible APIs
#### 1. You can feed the model a list of Numpy arrays as inputs
#### 2. Feed it a dictionary that maps input names to Numpy arrays

In [0]:
import numpy as np

In [0]:
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

In [0]:
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)